In this chapter, we will cover the following topics:

<ul>
    <li>Loading a DEM USGS ACSII CDED into PostGIS</li>
    <li>Creating an elevation profile</li>
    <li>Creating a hillshade raster from your DEM with ogr</li>
    <li>Generating slope and aspect images from your DEM</li>
    <li>Merging rasters to generate a color relief map</li>
    </ul>

## Introduction

Raster analysis works similar to vector analysis but the spatial relation is determined by the position of the raster cell. Most of our raster data is collected through diverse remote sensing techniques. In this chapter, the goals are quite simple and focused on working with and around a digital elevation model (DEM). The DEM we are using is from Whistler, BC, Canada, home to the 2010 Winter Olympics. Our DEM is in the form of the USGS ASCII CDED (.dem) format. The DEM is our source data that is used to derive several new raster datasets. As with other chapters, we will leverage Python as our glue to run scripts to enable a processing pipeline for raster data. The visualization of our data will play out with matplotlib along with the QGIS desktop GIS.

## 7.1. Loading a DEM USGS ACSII CDED into PostGIS

Importing and working with a DEM in PostGIS is what this recipe is all about. We begin our journey with a text file that's full of points and is stored in the USGS ASCII CDED format (to read more about the details of this format, feel free to look at the documentation page at http://www.gdal.org/frmt_usgsdem.html). The ASCII format is well known and accepted by many desktop GIS applications as a direct data source. Feel free to simply open up your ASCII file with QGIS to view the files and see the resulting raster representation that it creates for you. Our task at hand is to import this DEM file into a PostGIS database, creating a new PostGIS raster dataset within PostGIS We perform this task by using a command-line tool called raster2pgsql, which is installed along with your PostGIS installation. The raster2pgsql tool is located on Windows under C:\Program Files\PostgreSQL\9.3\bin\ if you are running PostgreSQL 9.

### Getting ready

Your data is available in the ch07/geodata/dem_3857.dem folder. Feel free to get the original DEM from GeoGratis Canada, the area around Whistler Mountain, British Columbia, at http://ftp2.cits.rncan.gc.ca/pub/geobase/official/cded/50k_dem/092/092j02.zip.

If you have not already created your Postgresql database in Chapter 1, Setting Up Your Geospatial Python Environment, do so now and then continue with starting your virtual environment to run this script.

Also, make sure that the raster2pgsql command is available in your command prompt. If not, set up your environment variables on Windows or a sym link on your Linux machine.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import subprocess
import psycopg2

db_host = "localhost"
db_user = "pluto"
db_passwd = "secret"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user,
                        port=db_port, password=db_passwd,
                        database=db_database)

# create a cursor
cur = conn.cursor()

# input USGS ASCII DEM (and CDED)
input_dem = "../geodata/dem_3857.dem"

# create an sql file for loading into the PostGIS database raster
# command line with options
# -c create new table
# -I option will create a spatial GiST index on the raster column
# -C will apply raster constraints
# -M vacuum analyse the raster table

command = 'raster2pgsql -c -C -I -M ' + input_dem + ' geodata.dem_3857'

# write the output to a file

temp_sql_file = "temp_sql.sql"

# open, create new file to write sql statements into
with open(temp_sql_file, 'wb') as f:
    try:
        result = subprocess.call(command, stdout=f, shell=True)
        if result != 0:
            raise Exception('error code %d' % result)

    except Exception as e:
        print(e)

# open the file full of insert statements created by raster2pgsql
with open(temp_sql_file, 'r') as r:
    # run through and execute each line inside the temp sql file
    for sql_insert in r:
        cur.execute(sql_insert)

print("please open QGIS >= 2.8.x and view your loaded DEM data")

### How it works...

Python, once again, is our glue that leverages the power of a command-line tool to do the dirty work. This time around, we use the Python subprocess module to call raster2pgsql the command-line tool. The psycopg2 module then executes our insert statements.

Starting from the top and working our way down, we see the database connection settings for psycopg2. The input path to our DEM is set as the input_dem variable. Then, we pack our command-line arguments into a single string called command. This is then run by subprocess. The individual command-line arguments are described in the code comments and further information and options can be found directly at http://postgis.refractions.net/docs/using_raster.xml.html#RT_Raster_Loader.

Now that the command is ready, we need to create a temporary file to store the generated SQL insert and create statements that the raster2pgsql command creates. Using the with open() syntax, we create our temporary file and then call the command using subprocess. We use stdout to specify where to write out this file. The shell=True argument comes with a big warning.

If all goes well, no exceptions should pop up, but if they do, we catch them using the standard Python try statement.

The last step is to open the newly created SQL file that's full of inserts and execute each line in the file using psycopg2. This populates our new table that has the name of the input DEM file.

Go ahead and open up QGIS | 2.8.x and have a look at the raster you've just loaded into PostGIS.

## 7.2. Creating an elevation profile

Creating an elevation profile is very helpful when trying to visualize a 3D terrain cross-section or simply to see the elevation gain of a bike tour. In this example, we will define our own LineString geometry and extract the elevation values from the DEMs that are located every 20 m along our line. The analysis will generate a new CSV file that we can open in Libre Office Calc or Microsoft Excel to visualize the new data as a line chart.

The 2D view of our line plotted on top of the elevation model as seen inside QGIS looks like this:

<img src="./50790OS_07_01.jpg" height=500 width=400>

This recipe calls for GDAL and Shapely. Make sure that you have them installed and are running them inside your python virtual environment that you set up earlier. To visualize your final CSV file, you must also install Libre Office Calc or some other charting software. The code to execute this is located at /ch07/code/ch07-02_elev_profile.py.

### How to do it...

Running the script directly from your command line will generate your CSV, so read the code comments to see all the little details of what is going on in order to generate our new file as follows:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys, gdal, os
from gdalconst import GA_ReadOnly
from os.path import realpath
from shapely.geometry import LineString


def get_elevation(x_coord, y_coord, raster, bands, gt):
    """
    get the elevation value of each pixel under
    location x, y
    :param x_coord: x coordinate
    :param y_coord: y coordinate
    :param raster: gdal raster open object
    :param bands: number of bands in image
    :param gt: raster limits
    :return: elevation value of raster at point x,y
    """
    elevation = []
    xOrigin = gt[0]
    yOrigin = gt[3]
    pixelWidth = gt[1]
    pixelHeight = gt[5]
    px = int((x_coord - xOrigin) / pixelWidth)
    py = int((y_coord - yOrigin) / pixelHeight)
    for j in range(bands):
        band = raster.GetRasterBand(j + 1)
        data = band.ReadAsArray(px, py, 1, 1)
        elevation.append(data[0][0])
    return elevation


def write_to_csv(csv_out,result_profile_x_z):
    # check if output file exists on disk if yes delete it
    if os.path.isfile(csv_out):
        os.remove(csv_out)
   
    # create new CSV file containing X (distance) and Z value pairs
    with open(csv_out, 'a') as outfile:
        # write first row column names into CSV
        outfile.write("distance,elevation" + "\n")
        # loop through each pair and write to CSV
        for x, z in result_profile_x_z:
            outfile.write(str(round(x, 2)) + ',' + str(round(z, 2)) + '\n')
           

if __name__ == '__main__':
    # set directory
    in_dem = realpath("../geodata/dem_3857.dem")
   
    # open the image
    ds = gdal.Open(in_dem, GA_ReadOnly)
   
    if ds is None:
        print 'Could not open image'
        sys.exit(1)
   
    # get raster bands
    bands = ds.RasterCount
   
    # get georeference info
    transform = ds.GetGeoTransform()

    # line defining the the profile
    line = LineString([(-13659328.8483806, 6450545.73152317), (-13651422.7820022, 6466228.25663444)])
    # length in meters of profile line
    length_m = line.length
   
    # lists of coords and elevations
    x = []
    y = []
    z = []
    # distance of the topographic profile
    distance = []
    for currentdistance in range(0, int(length_m), 20):
        # creation of the point on the line
        point = line.interpolate(currentdistance)
        xp, yp = point.x, point.y
        x.append(xp)
        y.append(yp)
        # extraction of the elevation value from the MNT
        z.append(get_elevation(xp, yp, ds, bands, transform)[0])
        distance.append(currentdistance)  
   
    print (x)
    print (y)
    print (z)
    print (distance)
   
    # combine distance and elevation vales as pairs
    profile_x_z = zip(distance,z)
   
    csv_file = os.path.realpath('../geodata/output_profile.csv')
    # output final csv data
    write_to_csv(csv_file, profile_x_z)

### How it works...

There are two functions that are used to create our elevation profile. The first get_elevation()function returns a single elevation value per pixel for each band in a raster. This means that our input raster can contain multiple bands of data. Our second function will write our results to a CSV file.

The get_elevation() function creates a list of elevation values; to achieve this, we need to extract some details from our input elevation raster. The x and y origin coordinates are used in combination with the raster pixel width and height to help find pixels in our raster. This information is then processed with our input x and y coordinates where we want the elevation value to be extracted.

Next up, we loop through all the available bands in our raster and find the elevation value per band that's located at coordinates x and y from our input. The ReadAsArray GDAL function finds this location, and then all we need to do is get the first object of the second nested list array. This value is then appended to a new list of elevation values.

To process our data, we define the input paths of our raster with the os.path.realpath() Python function that returns the full path to our input. GDAL is used to open our DEM raster and return the number of bands plus the x origin, y origin, pixel width, and pixel height information from our raster. This is located in the transform variable that's passed into our get_elevation() function.

Working our way further, we define our input LineString. This LineString defines where the cross-section profile is going to be extracted. To process our data, we want to extract elevation values every 20 m along our input LineString. This is done in the for loop as we specify the range based on the LineString length and our input of 20 m. Using the Shapely Interpolate linear referencing function, we then create a point object every 20 m. These values are then stored in separate x, y, and z lists, which are then updated. The z list contains our list of new elevation points. Individual elevations are collected by specifying the first object in the list that's returned by our get_elevation() function.

To put all this together in a CSV file, we use the Python zip function to combine the distance values with the elevation values. This creates the final two columns of data, showing us the distance from the starting point of our LineString on the x-axis and the elevation value on the y-axis.

Visualizing the results is then easy in Libre Office Calc or Microsoft Excel. Go ahead and open up the output CSV file located in your /ch07/geodata/output_profile.csv folder and create a simple line chart:

<img src="./50790OS_07_02.jpg" height=400 width=400>

Your resulting chart should look similar to what is shown in the preceding screenshot.

To plot the graph using Libre Office Calc, see the following plotting options:

<img src="./50790OS_07_03.jpg" height=400 width=400>

## 7.3. Creating a hillshade raster from your DEM with ogr

Our DEM can be the basis for many types of derived raster datasets. One of these derivatives is the so called hillshade raster dataset. A hillshade raster represents a 2D view of 3D elevation data, assigning gray raster shades and giving them a 3D effect by enabling you to see the highs and lows of your terrain. The hillshade is a pure visualization helper to create a nice looking map and show relief on a 2D map.

The pure Python solution to creating a hillshade is written by Roger Veciana i Rovira and you can find it at http://geoexamples.blogspot.co.at/2014/03/shaded-relief-images-using-gdal-python.html. There is also a nice solution by Joel Lawhead in Chapter 7, Python and Elevation Data of Learning Geospatial Analysis with Python. For those of you looking for a detailed description of the hillshade from ESRI, check this page out at http://webhelp.esri.com/arcgisdesktop/9.3/index.cfm?TopicName=How%20Hillshade%20works. The gdaldem hillshade command-line tool will be used to generate the image to disk.

<img src="./50790OS_07_04.jpg" height=400 width=400>

### Getting ready

The prerequisites for this example require the gdal (osgeo), numpy, and matplotlib python libraries. Plus, you need to have downloaded the data folder for this book and have the /ch07/geodata folder available for read/write access. We are directly accessing our USGS ASCII CDED DEM .dem file on disk to render our hillshade, so be sure that you have this folder. The code execution will take place as usual from your /ch07/code/ folder that runs the ch07-03_shaded_relief.py python file.

### How to do it...

Our Python script will run through a few mathematical operations and call the gdaldem command-line tool to generate our output using the following steps:

1. The code contains some math that is not always easy to follow; the calculation of the greyscale values is determined by the elevation and its surrounding pixels, so read along:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from osgeo import gdal
from numpy import gradient
from numpy import pi
from numpy import arctan
from numpy import arctan2
from numpy import sin
from numpy import cos
from numpy import sqrt
import matplotlib.pyplot as plt
import subprocess


def hillshade(array, azimuth, angle_altitude):
    """
    :param array: input USGS ASCII DEM / CDED .dem
    :param azimuth: sun position
    :param angle_altitude: sun angle
    :return: numpy array
    """

    x, y = gradient(array)
    slope = pi/2. - arctan(sqrt(x*x + y*y))
    aspect = arctan2(-x, y)
    azimuthrad = azimuth * pi / 180.
    altituderad = angle_altitude * pi / 180.


    shaded = sin(altituderad) * sin(slope)\
     + cos(altituderad) * cos(slope)\
     * cos(azimuthrad - aspect)
    return 255*(shaded + 1)/2

ds = gdal.Open('../geodata/092j02_0200_demw.dem')
arr = ds.ReadAsArray()

hs_array = hillshade(arr, 90, 45)
plt.imshow(hs_array,cmap='Greys')
plt.savefig('../geodata/hillshade_whistler.png')
plt.show()

# gdal command line tool called gdaldem
# link  http://www.gdal.org/gdaldem.html
# usage:
# gdaldem hillshade input_dem output_hillshade
# [-z ZFactor (default=1)] [-s scale* (default=1)]"
# [-az Azimuth (default=315)] [-alt Altitude (default=45)]
# [-alg ZevenbergenThorne] [-combined]
# [-compute_edges] [-b Band (default=1)] [-of format] [-co "NAME=VALUE"]* [-q]


create_hillshade = '''gdaldem hillshade -az 315 -alt 45 ../geodata/092j02_0200_demw.dem ../geodata/hillshade_3857.tif'''

subprocess.call(create_hillshade)

### How it works...

The hillshade function calculates slope and aspect values for each cell as the input to calculate the shaded gray value. The azimuth variable defines the direction of light in degrees that hits our DEM. Inverting and playing with azimuth can lead to some effects, such as valleys looking like hills and hills looking like valleys. Our shaded variable holds the shade values as an array that we can plot with matplotlib.

Using the gdaldem command-line tool is definitely more robust and faster than the pure Python solution. With gdaldem, we create a new hillshade TIF file on disk that can open either with a local image viewer or can be drag-and-dropped into QGIS. QGIS will automatically stretch the gray values for you so that you will be able to see a nice representation of your hillshade.

## 7.4. Generating slope and aspect images from your DEM

Slope maps are very useful, for example, to help biologists identify habitat zones. Certain species only live in areas that are very steep—mountain goats, for instance. The slope raster can then quick identify potential habitat areas. To visualize this, we use QGIS to display our slope map, which will look similar to the following image. The area in white indicates the steeper area and the darker the color, the flatter the terrain:

<img  src="./50790OS_07_05.jpg" height=400 width=400>

Our aspect map displays the direction that the surface faces towards—such as north, east, south, and west—and this is expressed in degrees. In the screenshot, the orange area represents warm south-facing areas. The north-facing sides are cooler and are indicated in different hues of blues from our color spectrum. To achieve the colors, the QGIS singleband pseudocolor was classified into five continuous classes as shown in the following screenshot:

<img  src="./50790OS_07_06.jpg" height=400 width=400>

### Getting ready
Ensure that your /ch07/geodata folder is downloaded and the DEM 092j02_0200_demw.dem file from Whistler, BC, Canada, is available.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import subprocess

# SLOPE
# - To generate a slope map from any GDAL-supported elevation raster :
# gdaldem slope input_dem output_slope_map"
# [-p use percent slope (default=degrees)] [-s scale* (default=1)]
# [-alg ZevenbergenThorne]
# [-compute_edges] [-b Band (default=1)] [-of format] [-co "NAME=VALUE"]* [-q]

create_slope = '''gdaldem slope ../geodata/092j02_0200_demw.dem ../geodata/slope_w-degrees.tif '''

subprocess.call(create_slope)

# ASPECT
# - To generate an aspect map from any GDAL-supported elevation raster
# Outputs a 32-bit float raster with pixel values from 0-360 indicating azimuth :
# gdaldem aspect input_dem output_aspect_map"
# [-trigonometric] [-zero_for_flat]
# [-alg ZevenbergenThorne]
# [-compute_edges] [-b Band (default=1)] [-of format] [-co "NAME=VALUE"]* [-q]

create_aspect = '''gdaldem aspect ../geodata/092j02_0200_demw.dem ../geodata/aspect_w.tif '''

subprocess.call(create_aspect)

### How it works...

The gdaldem command-line tool is our workhorse once again and all we need to do is pass along our DEM and specify an output file. Inside the code, you'll see the arguments passed include -co compress=lzw, which reduces the size of the image dramatically. Our -p option states that we want the results in a percentage slope followed by the input DEM and our output file.

As for our gdaldem aspect raster, the same compression applies this time and no other arguments are needed to generate the aspect raster. To visualize the aspect raster, open it inside QGIS and assign it a color as described earlier in the introduction.

## 7.5/ Merging rasters to generate a color relief map

Generating a color relief raster is a one-liner with the gdaldem color-relief command line. If you want something that's a little more visually appealing, we will perform a combination of a slope, hillshade, and some color relief. Our end result is a single new raster representing a merge of layers to give a nice visual effect of elevation relief. The results are going to look similar to the following image:

<img src="./50790OS_07_07.jpg" height=400 width=400>

### Getting ready

For this exercise, you need to have the GDAL libraries installed that include the gdaldem command-line tool.

### How to do it...

1. Let's begin by extracting some key information out of our DEM using the gdalinfo\ch07\code>gdalinfo ../geodata/092j02_0200_demw.dem command-line tool as follows:

<pre>
<code>
Driver: USGSDEM/USGS Optional ASCII DEM (and CDED)
Files: ../geodata/092j02_0200_demw.dem
       ../geodata/092j02_0200_demw.dem.aux.xml
Size is 1201, 1201
Coordinate System is:
GEOGCS["NAD83",
    DATUM["North_American_Datum_1983",
        SPHEROID["GRS 1980",6378137,298.257222101,
            AUTHORITY["EPSG","7019"]],
        TOWGS84[0,0,0,0,0,0,0],
        AUTHORITY["EPSG","6269"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9108"]],
    AUTHORITY["EPSG","4269"]]
Origin = (-123.000104166666630,50.250104166666667)
Pixel Size = (0.000208333333333,-0.000208333333333)
Metadata:
  AREA_OR_POINT=Point
Corner Coordinates:
Upper Left  (-123.0001042,  50.2501042) (123d 0' 0.37"W, 50d15' 0.38"N)
Lower Left  (-123.0001042,  49.9998958) (123d 0' 0.37"W, 49d59'59.63"N)
Upper Right (-122.7498958,  50.2501042) (122d44'59.62"W, 50d15' 0.38"N)
Lower Right (-122.7498958,  49.9998958) (122d44'59.62"W, 49d59'59.63"N)
Center      (-122.8750000,  50.1250000) (122d52'30.00"W, 50d 7'30.00"N)
Band 1 Block=1201x1201 Type=Int16, ColorInterp=Undefined
  Min=348.000 Max=2885.000
  Minimum=348.000, Maximum=2885.000, Mean=1481.196, StdDev=564.262
  NoData Value=-32767
  Unit Type: m
  Metadata:
    STATISTICS_MAXIMUM=2885
    STATISTICS_MEAN=1481.1960280116
    STATISTICS_MINIMUM=348
    STATISTICS_STDDEV=564.26229690401
</code>
</pre>

2. This key information is then used to create our color ramp.txt file. Start off by creating a new text file called ramp.txt and type in the following color codes:

<pre>
<code>
-32767 255 255 255
0 46 154 88
360 251 255 128
750 96 108 31
1100 148 130 55
2900 255 255 255
</code>
</pre>

3. The -32767 value defines our NODATA values in the white (255 255 255) RGB color. Now, save the ramp.txt file in the same folder as the following code that will generate the new raster color relief:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import subprocess

dem_file = '../geodata/092j02_0200_demw.dem'
hillshade_relief = '../geodata/hillshade.tif'
relief = '../geodata/relief.tif'
final_color_relief = '../geodata/final_color_relief.tif'

create_hillshade = 'gdaldem hillshade -co compress=lzw -compute_edges ' + dem_file +  ' ' + hillshade_relief
subprocess.call(create_hillshade, shell=True)
print create_hillshade

cr = 'gdaldem color-relief -co compress=lzw ' + dem_file + ' ramp.txt ' + relief
subprocess.call(cr)
print cr

merge = 'python hsv_merge.py ' + relief + ' ' + hillshade_relief + ' ' + final_color_relief
subprocess.call(merge)
print merge

create_slope = '''gdaldem slope -co compress=lzw ../geodata/092j02_0200_demw.dem ../geodata/slope_w-degrees.tif '''

subprocess.call(create_slope)

### How it works...

We need to chain together some commands and variables to get the desired results to look good. To begin our journey, we'll extract some key information from our DEM to enable us to create a color ramp that defines what colors are assigned to the elevation values. This new ramp.txt file stores our color ramp values that are then used by the gdaldem color-relief command.

The code then begins by defining the input and output needed as variables throughout this script. In the preceding code we have defined the input DEM and three output .tif files.

The first call will execute the gdaldem hillshade command to generate our hillshade. This is closely followed by the gdaldem color-relief command, creating our nice color raster that's based on the ramp.txt file we've defined. The ramp.txt file contains the NODATA value and sets it as the white RGB color. The five categories are based on the DEM data itself.

The final merge takes place using the Frank Warmerdam hsv_merge.py script that combines our relief output with the generated hillshade raster, leaving us with our final raster. Our result is a nice looking combination of a color-relief map and a hillshade.